In [ ]:
# pip install nltk 
# from the pop up window install: punkt, Averaged perceptron Tagger
# nltk.download()

In [ ]:
#pip install spacy

In [1]:
import json
import nltk
from nltk.tag import StanfordNERTagger
import os
import spacy
import textacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gensim.models import word2vec
from sklearn.cluster import KMeans
from sklearn import cluster, metrics

In [61]:
tickers= pd.read_json('/Users/macbookpro/Documents/GitHub/nlp/data/word_vectors/SP500_Tickers.json', typ='series')


url2='/Users/macbookpro/Documents/GitHub/nlp/data/word_vectors/article_headers_with_dates.txt'
url3 = '/Users/macbookpro/Documents/GitHub/nlp/parsed_scrapes/main.csv'
header = pd.read_csv(url3,sep=",")
header


,Unnamed: 0,Headline,Date,Provider
0,0,"​ UDAY states see Rs 11,989-crore drop in inte...",20/03/17,Economic Times India
1,1,Income Tax India: 1 taxpayer owes 11% of India...,24/01/17,Economic Times India
2,2,$10 billion unhedged gap in foreign exchange d...,17/10/16,Economic Times India
3,3,10% cost reduction can add $5.5 billion to Ind...,19/01/17,Economic Times India
4,4,10% cost reduction can add $5.5 billion to Ind...,19/01/17,Economic Times India
...,...,...,...,...
41770,43731,Why bond prices could collapse - MoneyWeek,20/10/08,Money Week
41771,43732,Why is the carry trade so dangerous? - MoneyWeek,24/08/07,Money Week
41772,43733,Why the dollar will get even stronger from her...,16/03/09,Money Week
41773,43734,New biofuel laws are utter madness - MoneyWeek,3/04/08,Money Week


In [62]:
ticker = tickers.to_frame('Ticker').reset_index()
ticker.columns=['Name','Ticker']
ticker

,Name,Ticker
0,AGILENT TECHNOLOGIES,A
1,AMERICAN AIRLINES,AAL
2,ADVANCE AUTO PARTS,AAP
3,APPLE,AAPL
4,ABBVIE,ABBV
...,...,...
495,XYLEM,XYL
496,YUM! BRANDS,YUM
497,ZIMMER BIOMET HOLDINGS,ZBH
498,ZIONS BANCORPORATION NA,ZION


In [95]:
list_ticker = ticker['Name'].to_list()

list_tick = ticker['Ticker'].to_list()
str3 = '|'.join(list_tick)
str1 = '|'.join(list_ticker)
str1 = str1 + str3
str1

"AGILENT TECHNOLOGIES|AMERICAN AIRLINES|ADVANCE AUTO PARTS|APPLE|ABBVIE|AMERISOURCEBERGEN|ABIOMED|ABBOTT LABORATORIES|ACCENTURE|ADOBE|ANALOG DEVICES|ARCHER-DANIELS-MIDLAND|AUTOMATIC DATA PROCESSING|ALLIANCE DATA SYSTEMS|AUTODESK|AMEREN|AMERICAN ELECTRIC POWER|AES|AFLAC|ALLERGAN|AMERICAN INTERNATIONAL|APARTMENT INVESTMENT & MANAGEMENT|ASSURANT|ARTHUR J GALLAGHER &|AKAMAI TECHNOLOGIES|ALBEMARLE|ALIGN TECHNOLOGY|ALASKA AIR|ALLSTATE|ALLEGION|ALEXION PHARMACEUTICALS|APPLIED MATERIALS|AMCOR|ADVANCED MICRO DEVICES|AMETEK|AFFILIATED MANAGERS|AMGEN|AMERIPRISE FINANCIAL|AMERICAN TOWER|AMAZONCOM|ARISTA NETWORKS|ANSYS|ANTHEM|AON|A O SMITH|APACHE|AIR PRODUCTS & CHEMICALS|AMPHENOL|APTIV|ALEXANDRIA REAL ESTATE EQUITIES|ARCONIC|ATMOS ENERGY|ACTIVISION BLIZZARD|AVALONBAY COMMUNITIES|BROADCOM|AVERY DENNISON|AMERICAN WATER WORKS|AMERICAN EXPRESS|AUTOZONE|BOEING|BANK OF AMERICA|BAXTER INTERNATIONAL|BB&T|BEST BUY|BECTON DICKINSON &|FRANKLIN RESOURCES|BROWN-FORMAN|BAKER HUGHES A GE|BIOGEN|BANK OF NEW YORK M

In [82]:
isin = pd.DataFrame(header['Headline'].str.contains(str1, regex=True))
isin

,Headline
0,True
1,True
2,True
3,True
4,True
...,...
41770,True
41771,True
41772,True
41773,True


In [83]:
index = isin[isin['Headline']==True].reset_index()['index']
#ticker.loc[index,'Ticker']
#ticker
index

0            0
1            1
2            2
3            3
4            4
         ...  
36103    41770
36104    41771
36105    41772
36106    41773
36107    41774
Name: index, Length: 36108, dtype: int64

In [84]:
index = isin[isin['Headline']==True].reset_index()['index']
index
header = header.loc[index,:].reset_index()#[['Date','Header']]

In [85]:
header


,index,Unnamed: 0,Headline,Date,Provider
0,0,0,"​ UDAY states see Rs 11,989-crore drop in inte...",20/03/17,Economic Times India
1,1,1,Income Tax India: 1 taxpayer owes 11% of India...,24/01/17,Economic Times India
2,2,2,$10 billion unhedged gap in foreign exchange d...,17/10/16,Economic Times India
3,3,3,10% cost reduction can add $5.5 billion to Ind...,19/01/17,Economic Times India
4,4,4,10% cost reduction can add $5.5 billion to Ind...,19/01/17,Economic Times India
...,...,...,...,...,...
36103,41770,43731,Why bond prices could collapse - MoneyWeek,20/10/08,Money Week
36104,41771,43732,Why is the carry trade so dangerous? - MoneyWeek,24/08/07,Money Week
36105,41772,43733,Why the dollar will get even stronger from her...,16/03/09,Money Week
36106,41773,43734,New biofuel laws are utter madness - MoneyWeek,3/04/08,Money Week


In [100]:
#extraction = list(textacy.extract.subject_verb_object_triples(example))


nlp = spacy.load('en_core_web_sm')

#header = 'Goldman Sachs announce purchase of Microsoft'

# Enter your username in here. I've put the paths you wrote as an attribute in the dictionary below
user = 'Francesco'

# Dictionary containing the Java paths for each user
java_paths = {
    'Conor': '/usr/lib/jvm/java-8-openjdk-amd64',
    'Francesco': '/Users/macbookpro/Downloads/jdk-13.0.1.jdk/Contents/Home/bin/java',
    'Cecilia': 'C:/Program Files/Java/jdk-13.0.1/bin/java.exe'
}
os.environ['JAVAHOME'] = java_paths[user]

st = StanfordNERTagger(
    '/Users/macbookpro/Documents/GitHub/nlp/ner/stanford-ner-2014-06-16/classifiers/english.all.3class.distsim.crf.ser.gz',
    '/Users/macbookpro/Documents/GitHub/nlp/ner/stanford-ner-2014-06-16/stanford-ner.jar',
    encoding = 'utf-8'
)
COMPANY_TYPES = ['PERSON', 'ORGANIZATION']

def get_ticker1(name, tickers):
    for key, val in tickers.items():
        if val.upper() == name.upper():
            return f'__{val}'
        if name.upper() in key.upper():
            return f'__{val}'
    return None

# def parse_header(header, max_company_name_length = max((len(x) for x in tickers))):
def parse_header1(header,tickers):
    '''Attempt to replace all organisations in a header with their ticker'''
    header = header.lower()
    nlp_header = nlp(header)

    tokens = list(textacy.extract.subject_verb_object_triples(nlp_header))  
    parsed_words = []
    for i in range(len(tokens)):
        ticker_0 = get_ticker1(str(tokens[i][0]), tickers)
        ticker_2 = get_ticker1(str(tokens[i][2]), tickers)  
        sub = str(tokens[i][0])
        obj = str(tokens[i][2])
        tick_sub = ''
        tick_obj = ''
        if ticker_0 is not None:
            sub = ticker_0
            tick_sub = 'sub'
        if ticker_2 is not None:
            obj = ticker_2
            tick_obj = 'obj'
        verb = str(tokens[i][1])
        tick = tick_sub + tick_obj
        if (ticker_0 != None) | (ticker_2 != None):
            parsed_words.append([sub, verb, obj, tick])
        
    return parsed_words

In [101]:
print(len(header['Headline']))

svo_df = pd.DataFrame(columns = ['Date', 'Sub', 'Verb', 'Obj', 'Tick_Type'])

for i in range (len(header['Headline'])):
    #print(i)
    headline = header.loc[i,'Headline']
    date = header.loc[i,'Date']
    svo_headlines = parse_header1(headline,tickers)
    
    for ls in svo_headlines:
        svo_df.loc[len(svo_df)] =  [date] + ls

    
svo_df

36108


,Date,Sub,Verb,Obj,Tick_Type
0,17/10/16,__GPS,may put,rupee,sub
1,22/08/17,__PKI,may see,revenue loss,sub
2,30/06/17,__STI,may see,provisions,sub
3,28/04/18,discretion,to play,__MPC,obj
4,6/12/16,kg,seized,__GS,obj
...,...,...,...,...,...
4726,28/10/19,killer shark,takes,__HP,obj
4727,3/11/19,__AOS,needs,to be split,sub
4728,6/11/19,__IBM,employs,freelancers,sub
4729,31/10/19,__FDX,printing,moneyweek,sub


In [112]:
test = svo_df
svo_df['Date'] = pd.to_datetime(svo_df['Date'])

svo_df = svo_df.sort_values(by='Date')
svo_df
#print(type(test.loc[1,'Date']))

,Date,Sub,Verb,Obj,Tick_Type
3646,1970-01-01,government,to use,__A,obj
3645,1970-01-01,india,sends,__AAP,obj
4461,2006-01-05,__TPR,is strangling,moneyweek,sub
4730,2006-05-18,denmark,is leading,__BRK.B,obj
136,2006-08-26,holcim,fined,__AMD,obj
...,...,...,...,...,...
3531,2019-12-09,__LRCX,adds,outlook report,sub
3550,2019-12-09,spot,introduces,__ADI,obj
3492,2019-12-09,__A,announces,__BR,subobj
4718,2019-12-10,__AOS,could be powering,moneyweek,sub


In [ ]:
header['nlp'] = header['with ticker'].apply(nlp)

#extraction = list(textacy.extract.subject_verb_object_triples(example))